In [183]:
import pandas as pd
from keras.layers import Input, Embedding, LSTM, Dense, Dot, Softmax, Concatenate, BatchNormalization, Attention
from keras.models import Model
import tensorflow as tf
import numpy as np
from keras.utils import pad_sequences

In [184]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'orders_and_products_by_customer.pkl')

In [185]:
product_data = product_data.sample(n=1000)
product_data

,customer_id,order_ids,product_ids,product_names,amounts,total_prices,order_dates
127015,6243591.0,[6243587],[[5338874]],[[Riemchensandale Rauleder gelb]],[[1]],[[99.95]],[[1624357547]]
56208,4554290.0,[4554288],[[3481950]],[[Slipper Glattleder braun]],[[1]],[[116.97]],[[1604230026]]
119167,6011993.0,[6011990],[[4913257]],[[Sneaker low Glattleder weiß]],[[1]],[[89.0]],[[1622568756]]
101862,5625543.0,[5625536],"[[5150425, 4822026]]","[[Sportliche Ballerina Rauleder blau, Sportlic...","[[1, 1]]","[[99.95, 99.95]]","[[1618493047, 1618493047]]"
15135,2122541.0,[10857219],"[[10230096, 10231865]]","[[Slipper Glattleder schwarz, Slipper Rauleder...","[[1, 1]]","[[130.0, 130.0]]","[[1677142076, 1677142076]]"
...,...,...,...,...,...,...,...
56231,4554729.0,[4554725],[[4234484]],[[Sneaker high Glattleder schwarz]],[[1]],[[146.22]],[[1604234226]]
238156,8916562.0,[8916559],[[7590576]],[[Plateau Pumps Textil silber]],[[1]],[[89.95]],[[1655485645]]
99266,5565036.0,[5565034],[[5206785]],[[Slipper Rauleder blau]],[[1]],[[99.95]],[[1617895847]]
36198,4065663.0,[4217044],[[3842446]],[[Ankle Boot Rauleder braun]],[[1]],[[97.43]],[[1601483526]]


In [186]:
product_data

,customer_id,order_ids,product_ids,product_names,amounts,total_prices,order_dates
127015,6243591.0,[6243587],[[5338874]],[[Riemchensandale Rauleder gelb]],[[1]],[[99.95]],[[1624357547]]
56208,4554290.0,[4554288],[[3481950]],[[Slipper Glattleder braun]],[[1]],[[116.97]],[[1604230026]]
119167,6011993.0,[6011990],[[4913257]],[[Sneaker low Glattleder weiß]],[[1]],[[89.0]],[[1622568756]]
101862,5625543.0,[5625536],"[[5150425, 4822026]]","[[Sportliche Ballerina Rauleder blau, Sportlic...","[[1, 1]]","[[99.95, 99.95]]","[[1618493047, 1618493047]]"
15135,2122541.0,[10857219],"[[10230096, 10231865]]","[[Slipper Glattleder schwarz, Slipper Rauleder...","[[1, 1]]","[[130.0, 130.0]]","[[1677142076, 1677142076]]"
...,...,...,...,...,...,...,...
56231,4554729.0,[4554725],[[4234484]],[[Sneaker high Glattleder schwarz]],[[1]],[[146.22]],[[1604234226]]
238156,8916562.0,[8916559],[[7590576]],[[Plateau Pumps Textil silber]],[[1]],[[89.95]],[[1655485645]]
99266,5565036.0,[5565034],[[5206785]],[[Slipper Rauleder blau]],[[1]],[[99.95]],[[1617895847]]
36198,4065663.0,[4217044],[[3842446]],[[Ankle Boot Rauleder braun]],[[1]],[[97.43]],[[1601483526]]


In [187]:
display(product_data[['customer_id', 'order_ids', 'product_ids', 'amounts', 'order_dates']])

,customer_id,order_ids,product_ids,amounts,order_dates
127015,6243591.0,[6243587],[[5338874]],[[1]],[[1624357547]]
56208,4554290.0,[4554288],[[3481950]],[[1]],[[1604230026]]
119167,6011993.0,[6011990],[[4913257]],[[1]],[[1622568756]]
101862,5625543.0,[5625536],"[[5150425, 4822026]]","[[1, 1]]","[[1618493047, 1618493047]]"
15135,2122541.0,[10857219],"[[10230096, 10231865]]","[[1, 1]]","[[1677142076, 1677142076]]"
...,...,...,...,...,...
56231,4554729.0,[4554725],[[4234484]],[[1]],[[1604234226]]
238156,8916562.0,[8916559],[[7590576]],[[1]],[[1655485645]]
99266,5565036.0,[5565034],[[5206785]],[[1]],[[1617895847]]
36198,4065663.0,[4217044],[[3842446]],[[1]],[[1601483526]]


In [188]:
product_data.order_ids

127015     [6243587]
56208      [4554288]
119167     [6011990]
101862     [5625536]
15135     [10857219]
             ...    
56231      [4554725]
238156     [8916559]
99266      [5565034]
36198      [4217044]
161425     [7028154]
Name: order_ids, Length: 1000, dtype: object

In [189]:
print(product_data.product_ids, product_data.order_dates)

127015               [[5338874]]
56208                [[3481950]]
119167               [[4913257]]
101862      [[5150425, 4822026]]
15135     [[10230096, 10231865]]
                   ...          
56231                [[4234484]]
238156               [[7590576]]
99266                [[5206785]]
36198                [[3842446]]
161425      [[6435988, 6113895]]
Name: product_ids, Length: 1000, dtype: object 127015                [[1624357547]]
56208                 [[1604230026]]
119167                [[1622568756]]
101862    [[1618493047, 1618493047]]
15135     [[1677142076, 1677142076]]
                     ...            
56231                 [[1604234226]]
238156                [[1655485645]]
99266                 [[1617895847]]
36198                 [[1601483526]]
161425    [[1633418531, 1633418531]]
Name: order_dates, Length: 1000, dtype: object


In [190]:
def bringToSameSizeOneDim(column):
    max_length = max(column.apply(len))
    return column.apply(lambda x: x + [0] * (max_length - len(x)))


def bringToSameSizeTwoDimAndFlatten(column):
    max_length = max(column.apply(lambda x: max(len(sublist) for sublist in x)))
    new_column = column.apply(lambda x: [sublist + [0] * (max_length - len(sublist)) for sublist in x])
    new_column = new_column.apply(lambda x: x + [[0] * max_length] * (max_length - len(x)))
    return new_column.apply(lambda x: [item for sublist in x for item in sublist])

product_data.order_ids = bringToSameSizeOneDim(product_data.order_ids)
product_data.product_ids = bringToSameSizeTwoDimAndFlatten(product_data.product_ids)
product_data.amounts = bringToSameSizeTwoDimAndFlatten(product_data.amounts)
product_data.total_prices = bringToSameSizeTwoDimAndFlatten(product_data.total_prices)
product_data.order_dates = bringToSameSizeTwoDimAndFlatten(product_data.order_dates)




In [191]:
def convertToSequence(column):
    list = column.tolist()
    max_length = max(len(x) for x in list)
    return pad_sequences(list, maxlen=max_length, padding='post', value=0), max_length

products_inputs, max_length_products = convertToSequence(product_data.product_ids)
dates_inputs, max_length_dates = convertToSequence(product_data.order_dates)
target_data, _ = convertToSequence(product_data.amounts)




In [192]:
max_length_dates, dates_inputs.shape[1], target_data.shape

(70, 70, (1000, 70))

In [193]:
# Define the model architecture
# num_orders = sum(len(order) for order in product_data.order_ids)
# num_products =  sum(len(order) for order in product_data.product_ids)
# num_units = len([amount for order_amounts in product_data.amounts for amount in order_amounts])
# num_prices = len([amount for order_amounts in product_data.total_prices for amount in order_amounts])
# num_dates = len([amount for order_amounts in product_data.order_dates for amount in order_amounts])
#
# print(num_orders, num_products, num_units, num_prices, num_dates)

In [194]:
num_dates = len(np.unique(product_data.order_dates))
num_products = len(np.unique(product_data.product_ids))

In [195]:
from keras.layers import Reshape

date_inputs = Input(shape=(max_length_dates,))
product_inputs = Input(shape=(max_length_products,))

print(dates_inputs.shape)
print(products_inputs.shape)

encoding_padding_mask = tf.math.logical_not(tf.math.equal(product_inputs, 0))

# Embedding layers
date_emb = Embedding(num_dates, 16, input_length=max_length_dates)(date_inputs)
product_emb = Embedding(num_products, 16, input_length=max_length_products)(product_inputs)

concat_embedding_input = Concatenate(
    axis=-1, name='concat_embedding_input')([date_emb, product_emb])

batchnorm_inputs = BatchNormalization(
    name='batchnorm_inputs')(concat_embedding_input)
# LSTM layer

# reshaped_input = Reshape((max_length, -1))(concat_embedding_input)

# reshaped_inputs = Reshape((max_length_dates, 7))(concat_embedding_input)
lstm = LSTM(64, return_sequences=True)(concat_embedding_input)

lstm = BatchNormalization(name='batchnorm_lstm')(lstm)

att = Attention(use_scale=False,
                name='attention')(inputs=[lstm, lstm],
                                  mask=[encoding_padding_mask,
                                        encoding_padding_mask])


# Concatenate layer
#concat = Concatenate(axis=2)([context, price_inputs])

# Dense layer
output = Dense(target_data.shape[1], activation = 'softmax')(att)
# Define the model
model = Model(inputs=[date_inputs, product_inputs], outputs=output)
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
model.fit(x=[dates_inputs, products_inputs], y=target_data, epochs=10, batch_size=32)  # Adjust epochs and batch size as needed


(1000, 70)
(1000, 70)
Epoch 1/10


2023-05-29 14:01:02.391058: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


InvalidArgumentError: Graph execution error:

Detected at node 'model_29/embedding_74/embedding_lookup' defined at (most recent call last):
    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/thomashuber/miniconda3/envs/m1-mast

In [ ]:
# Input layers
date_inputs = Input(shape=dates_inputs.shape)
# product_inputs = Input(shape=(num_orders, num_units))
product_inputs = Input(products_inputs.shape)
# price_inputs = Input(shape=(num_orders, num_prices))

encoding_padding_mask = tf.math.logical_not(tf.math.equal(product_inputs, 0))

# Embedding layers
date_emb = Embedding(num_dates, 16, input_length=num_dates)(date_inputs)
product_emb = Embedding(num_products, 16, input_length=num_units)(product_inputs)

concat_embedding_input = Concatenate(
    name='concat_embedding_input')([date_emb, product_emb])

concat_embedding_input = BatchNormalization(
    name='batchnorm_inputs')(concat_embedding_input)
# LSTM layer

lstm = LSTM(64, return_sequences=True)(concat_embedding_input)

lstm = BatchNormalization(name='batchnorm_lstm')(lstm)

att = Attention(use_scale=False,
                                name='attention')(inputs=[lstm, lstm],
                                                  mask=[encoding_padding_mask,
                                                        encoding_padding_mask])


# Concatenate layer
#concat = Concatenate(axis=2)([context, price_inputs])

# Dense layer
output = Dense(1)(att)
# Define the model
model = Model(inputs=[date_inputs, product_inputs], outputs=output)
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

In [ ]:
# Fit the model
model.fit(x=[dates_inputs, products_inputs], y=target_data, epochs=10, batch_size=32)  # Adjust epochs and batch size as needed


In [ ]:
#display(X_train)
#output

# product_ids_flat = product_data.product_ids.to_numpy().reshape(-1)
# order_dates_flat = product_data.order_dates.to_numpy().reshape(-1)

# Create the output variable
output_variable = product_data.amounts.to_numpy().reshape(-1, 1)

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((product_data.order_dates.to_numpy(), product_data.product_ids.to_numpy(), output_variable))

# Fit the model
history = model.fit(dataset, epochs=50, batch_size=32)


In [ ]:
display(order_dates_flat)

In [ ]:
y = []
output_variable = product_data.amounts.to_numpy().reshape(-1, 1)
dataset = tf.data.Dataset.from_tensor_slices((np.array(product_data.order_dates), np.array(product_data.product_ids), output_variable))
history = model.fit(dataset, epochs=50, batch_size=32)

In [ ]:
# Evaluate the model
loss, mse = model.evaluate([product_data.order_dates_test, product_data.product_ids_test, product_data.total_prices_test], product_data.total_prices_test)
print("Test loss: {:.4f}, Test MSE: {:.4f}".format(loss, mse))

In [ ]:
# split the data into training and testing sets
#X = pd.get_dummies(product_data['product__id'], prefix='product')
#y = product_data['amount']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)